# Carica dati storici

Carica i dati Parquet per un ticker e separa EOD e intraday.

In [1]:
import sys
import polars as pl
from pathlib import Path

sys.path.insert(0, str(Path("../../").resolve()))

from engine.datafeed import DataFeed, MarketData

In [2]:
# Scegli il ticker
ticker = "SX5E"

# Path relativo alla root del progetto (2 livelli sopra da strategies/example/)
data_dir = Path("../../database") 


In [3]:
# PRICES_PATH = str(project_root / 'data')
universe = ['SX5E']
start_date='2025-01-01'
end_date='2025-12-31'

# cfg = Config(universe=universe, start_date=start_date, end_date=end_date, frequency="eod")
feed = DataFeed(prices_base=data_dir)

# DataFeed DataFrame

In [4]:
# Carica i dati di mercato usando DataFeed
df = feed.get_market_data(start_date, end_date, 'eod', universe)
print(f"Shape: {df.shape}")
print(f"\nPrime righe:")
print(df.head())


Shape: (256, 10)

Prime righe:
shape: (5, 10)
┌────────────┬──────────┬────────┬─────────┬───┬─────────┬──────────────┬────────────────┬──────┐
│ date       ┆ time     ┆ ticker ┆ open    ┆ … ┆ close   ┆ volume       ┆ insertion_time ┆ type │
│ ---        ┆ ---      ┆ ---    ┆ ---     ┆   ┆ ---     ┆ ---          ┆ ---            ┆ ---  │
│ date       ┆ time     ┆ str    ┆ f64     ┆   ┆ f64     ┆ f64          ┆ datetime[μs]   ┆ str  │
╞════════════╪══════════╪════════╪═════════╪═══╪═════════╪══════════════╪════════════════╪══════╡
│ 2025-01-02 ┆ 01:00:00 ┆ SX5E   ┆ 4890.54 ┆ … ┆ 4917.88 ┆ 2.386088e8   ┆ 2026-02-03     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 14:19:00       ┆      │
│ 2025-01-03 ┆ 01:00:00 ┆ SX5E   ┆ 4915.54 ┆ … ┆ 4871.45 ┆ 2.06090877e8 ┆ 2026-02-03     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 14:19:00       ┆      │
│ 2025-01-06 ┆ 01:00:00 ┆ SX5E   ┆ 4893.32 ┆ … ┆ 4986.64 ┆ 2.62450529e8 

In [5]:
feed.get_data_excel(start_date, end_date, '1m', universe)

'c:\\Users\\S542279\\Desktop\\quanto\\strategies\\example\\SX5E_2025-01-01_2025-12-31_1m.xlsx'

In [ ]:
# Crea MarketData dal DataFrame storico
market_data = MarketData.from_dataframe(df)
print(f"MarketData creato con {len(market_data.symbols())} simboli")
print(f"Current date: {market_data.current_date()}")
print(f"Current time: {market_data.current_time()}")


In [ ]:
# Esempi di accesso ai prezzi
print("Prezzo corrente:")
print(f"  SX5E close: ${market_data.price('SX5E', 'close'):.2f}")

print("\nPrezzo T-1 (1 bar fa):")
print(f"  SX5E close: ${market_data.price('SX5E', 'close', 1):.2f}")

print("\nUltimi 10 prezzi:")
last_10 = market_data.price('SX5E', 'close', slice(0, 10))
print(f"  Shape: {len(last_10)} valori")
print(f"  Ultimi 3: {last_10[-3:].to_list()}")
